In [ ]:
# Automatically install required packages if missing
import subprocess
import sys

def install_if_missing(package_name, import_name=None):
    import_name = import_name or package_name
    try:
        __import__(import_name)
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

# Install required packages
install_if_missing('pandas')
install_if_missing('numpy')
install_if_missing('matplotlib')
install_if_missing('ipywidgets')

# --- Interactive Module Code ---
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import pandas as pd
import numpy as np
from IPython.display import display


def launch_interactive_threshold_plot(eval_df_split):
    df = eval_df_split[eval_df_split['Sample'] == 'Combined'].copy()
    df['Method'] = np.where(
        df['Method'] == 'Cascade Fallback',
        'Heuristic Fallback',
        df['Method']
    )

    categories = sorted(df['Category'].unique())
    thresholds = sorted(df['Threshold'].unique())

    method_colors = {
        'Model Only': 'blue',
        'Heuristic Fallback': 'green'
    }

    def plot_for_threshold(category, threshold):
        subset = df[df['Category'] == category]

        fig, ax1 = plt.subplots(figsize=(10, 5))
        ax2 = ax1.twinx()

        metric_styles = {
            'Precision': '-',
            'Recall': '--',
            'False Omission Rate': ':'
        }

        results = []

        for method, method_df in subset.groupby('Method'):
            color = method_colors.get(method, 'gray')

            for metric, style in metric_styles.items():
                ax1.plot(method_df['Threshold'], method_df[metric],
                         linestyle=style,
                         color=color,
                         label=f'{method} - {metric}')

            closest_idx = (method_df['Threshold'] - threshold).abs().idxmin()
            row = method_df.loc[closest_idx]

            ax1.annotate(f"P: {row['Precision']:.2f}", xy=(threshold, row['Precision']), xytext=(5, -15),
                         textcoords='offset points', fontsize=10, color=color)
            ax1.annotate(f"R: {row['Recall']:.2f}", xy=(threshold, row['Recall']), xytext=(5, 5),
                         textcoords='offset points', fontsize=10, color=color)
            ax1.annotate(f"O: {row['False Omission Rate']:.2f}", xy=(threshold, row['False Omission Rate']), xytext=(5, 5),
                         textcoords='offset points', fontsize=10, color=color)

            if method == 'Model Only':
                ax2.plot(method_df['Threshold'], method_df['Not Classified Count'],
                         color='red', linestyle='-', label=f'{method} - Not Classified Count')
                ax2.annotate(f"N: {int(row['Not Classified Count'])}", xy=(threshold, row['Not Classified Count']), xytext=(5, -10),
                             textcoords='offset points', fontsize=9, color='red')

            results.append({
                'Category': category,
                'Method': method,
                'Threshold': row['Threshold'],
                'Precision': row['Precision'],
                'Recall': row['Recall'],
                'False Omission Rate': row['False Omission Rate'],
                'Miss Rate': row['Miss Rate'],
                'Not Classified Count': row['Not Classified Count'],
                'Not Classified Rate': row['Not Classified Rate']
            })

        ax1.axvline(threshold, color='black', linestyle='-', alpha=0.3)
        ax1.set_title(f'{category} - Metrics by Threshold')
        ax1.set_xlabel('Threshold')
        ax1.set_ylabel('Score (P, R, O)')
        ax2.set_ylabel('Not Classified Count (Model Only)', color='red')

        ax1.set_ylim(0, 1.05)
        ax1.grid(True)

        lines_labels_1 = ax1.get_legend_handles_labels()
        lines_labels_2 = ax2.get_legend_handles_labels()
        ax1.legend(*[sum(lol, []) for lol in zip(lines_labels_1, lines_labels_2)], loc='upper left')

        plt.tight_layout()
        plt.show()

        display(pd.DataFrame(results).round(3))

    interact(
        plot_for_threshold,
        category=widgets.Dropdown(options=categories, value=categories[0], description='Category:'),
        threshold=widgets.FloatSlider(value=thresholds[0], min=min(thresholds), max=max(thresholds), step=0.001, description='Threshold:')
    )

# Load CSV
df = pd.read_csv('./eval_df_split.csv')
launch_interactive_threshold_plot(df)